<a href="https://colab.research.google.com/github/ALejandroCh17/Tennis-Ball-Robot/blob/daniel_branch/model3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load the Dataset

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
"""
base_dir = '/content/drive/My Drive/capstone_dataset_4/'

train_dir = '/content/drive/My Drive/capstone_dataset_4/train'
test_dir = '/content/drive/My Drive/capstone_dataset_4/test'
valid_dir = '/content/drive/My Drive/capstone_dataset_4/valid'
"""

"\nbase_dir = '/content/drive/My Drive/capstone_dataset_4/'\n\ntrain_dir = '/content/drive/My Drive/capstone_dataset_4/train'\ntest_dir = '/content/drive/My Drive/capstone_dataset_4/test'\nvalid_dir = '/content/drive/My Drive/capstone_dataset_4/valid'\n"

In [10]:
"""
import os

trainFileCount = len([name for name in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, name))])
testFileCount = len([name for name in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, name))])
validFileCount = len([name for name in os.listdir(valid_dir) if os.path.isfile(os.path.join(valid_dir, name))])

print("Train count: ", str(trainFileCount))
print("Test count: ", str(testFileCount))
print("Valid count: ", str(validFileCount))
print("Total: ", str(trainFileCount+testFileCount+validFileCount))
"""

'\nimport os\n\ntrainFileCount = len([name for name in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, name))])\ntestFileCount = len([name for name in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, name))])\nvalidFileCount = len([name for name in os.listdir(valid_dir) if os.path.isfile(os.path.join(valid_dir, name))])\n\nprint("Train count: ", str(trainFileCount))\nprint("Test count: ", str(testFileCount))\nprint("Valid count: ", str(validFileCount))\nprint("Total: ", str(trainFileCount+testFileCount+validFileCount))\n'

In [11]:
"""
#combining all the data
import shutil

#create new directory to store total set
totalDir = '/content/drive/My Drive/capstone_dataset_4/combined'
os.makedirs(totalDir, exist_ok=True)

#iterate over all 3 directories and copy files to totalDir
for directory in [train_dir, test_dir, valid_dir]:
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):
            shutil.copy2(os.path.join(directory, filename), os.path.join(totalDir, filename))
"""

"\n#combining all the data\nimport shutil\n\n#create new directory to store total set\ntotalDir = '/content/drive/My Drive/capstone_dataset_4/combined'\nos.makedirs(totalDir, exist_ok=True)\n\n#iterate over all 3 directories and copy files to totalDir\nfor directory in [train_dir, test_dir, valid_dir]:\n    for filename in os.listdir(directory):\n        if filename.endswith('.jpg'):\n            shutil.copy2(os.path.join(directory, filename), os.path.join(totalDir, filename))\n"

In [12]:
totalDir = '/content/drive/My Drive/capstone_dataset_4/combined'

In [13]:
#count the number of files in the new total directory
totalFileCount = len([name for name in os.listdir(totalDir) if os.path.isfile(os.path.join(totalDir, name))])

print("Total combined count: ", str(totalFileCount))

Total combined count:  2548


Removed 3 non images files, the JSON files

#Preprocessing

In [14]:
#check to see what size all the images by returning if they are all the same size.
import os
import cv2

def check_same_size(path):
  sizes = set()
  for filename in os.listdir(path):
    if filename.endswith('.jpg'):
      img = cv2.imread(os.path.join(path, filename))
      if img is None:
        print(f"Image not loaded successfully: {filename}")
        continue
      sizes.add(img.shape)

  if len(sizes) == 1:
    print("Yes, they are all the same size.")
  else:
    print("No, they are not all the same size.")

check_same_size(totalDir)


Yes, they are all the same size.


In [23]:
# prompt: get the dimensions of one image

import cv2

# Read the first image in the directory
image = cv2.imread(os.path.join(totalDir, os.listdir(totalDir)[0]))

# Get the dimensions of the image
height, width, channels = image.shape

# Print the dimensions
print(f"Image dimensions: {height}x{width}x{channels}")


Image dimensions: 640x640x3


In [15]:
#Normalize dataset to scale pixel values to a common scale
import cv2
import numpy as np

def normalize_image(image):

  mean = np.mean(image)
  std = np.std(image)

  normalized_image = (image - mean) / std

  return normalized_image

# Normalize all images in the combined directory
for filename in os.listdir(totalDir):
  if filename.endswith('.jpg'):
    image = cv2.imread(os.path.join(totalDir, filename))
    normalized_image = normalize_image(image)
    cv2.imwrite(os.path.join(totalDir, filename), normalized_image)


#Split the dataset

In [32]:
# prompt: split the dataset into 70/30 without creating a new directory. use sklearn

from sklearn.model_selection import train_test_split

# Get a list of all the image filenames in the combined directory
image_filenames = [filename for filename in os.listdir(totalDir) if filename.endswith('.jpg')]

# Split the filenames into train and test sets, with a 70/30 split
train_filenames, test_filenames = train_test_split(image_filenames, test_size=0.3, random_state=42)

# No need to create new directories, as the images are already in the combined directory
# You can now use the train_filenames and test_filenames lists to access the training and test images, respectively.


In [26]:
# prompt: create a deep learning model to train on the dataset. this is a one class classification task

# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define the model architecture
model = keras.Sequential([
  layers.Flatten(input_shape=(640, 640, 3)),
  layers.Dense(128, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

In [27]:
#Investigate model input shape
model.input_shape

(None, 640, 640, 3)

In [33]:
#investigate shpe of training data
train_filenames[0].shape

AttributeError: 'str' object has no attribute 'shape'

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_filenames, epochs=10)

# Evaluate the model
model.evaluate(test_filenames)

# Save the model
model.save('my_model.h5')